In [1]:
import os

In [2]:
pwd

'd:\\Data Science\\NLP\\SentimentAnalysis\\research'

In [3]:
os.chdir("../")


In [4]:
pwd

'd:\\Data Science\\NLP\\SentimentAnalysis'

In [5]:
import os
from pathlib import Path
from typing import Optional
import yaml
from dataclasses import dataclass

### b.	To be updated entity > constructor file (__init__.py)

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    stopwords_path: Path

### c.	To be updated config > configuration.py file

In [7]:
from SentimentAnalysis.constants import *
from SentimentAnalysis.utils.common import read_yaml
from SentimentAnalysis.utils.common import create_directories

In [8]:
class ConfigurationManager:
    
    def __init__(
                self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH):
            

            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)

            create_directories([self.config.dataStore_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
          config = self.config.data_transformation

          create_directories([config.root_dir])
          
          data_transformation_config = DataTransformationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                stopwords_path=config.stopwords_path,      
          )     
          return data_transformation_config 

In [13]:
class ConfigurationManager:
    
    def __init__(
                self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH):
            

            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)

            create_directories([self.config.dataStore_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
          config = self.config.data_transformation

          create_directories([config.root_dir])
          

          data_transformation_config = DataTransformationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                stopwords_path=config.stopwords_path,
                
          )     
          return data_transformation_config

In [25]:
import os
from SentimentAnalysis.logging import logger
from datasets import load_dataset, load_from_disk


In [26]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelBinarizer
import pandas as pd 

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = ToktokTokenizer()
        self.stopwords_list = set(stopwords.words('english'))
        self.stemmer = SnowballStemmer(language='english')
        self.lb = LabelBinarizer()

    def html(self, review):
        # Remove HTML tags from the text
        soup = BeautifulSoup(review, "html.parser")
        return soup.get_text()

    def deEmojify(self, review):
        # Remove emojis from the text
        regrex_pattern = re.compile(pattern="["
                                      u"\U0001F600-\U0001F64F"  # emoticons
                                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                      "]+", flags=re.UNICODE)
        return regrex_pattern.sub(r'', review)

    def to_unicode(self, review):
        # Convert text to unicode
        if isinstance(review, float):
            review = str(review)
        if isinstance(review, int):
            review = str(review)
        if not isinstance(review, str):
            review = review.decode('utf-8', 'ignore')
        return review

    def remove_between_square_brackets(self, review):
        # Remove text between square brackets
        return re.sub(r'\[[^]]*\]', '', review)

    def remove_special_characters(self, review, remove_digits=True):
        # Remove special characters and optionally digits
        pattern = r'[^a-zA-z0-9\s]' if remove_digits else r'[^a-zA-z\s]'
        review = re.sub(pattern, '', review)
        return review

    def denoise_text(self, review):
        # Denoise the text using multiple cleaning steps
        review = self.to_unicode(review)
        review = self.html(review)
        review = re.sub(r"http\S+", "", review)
        review = self.deEmojify(review)
        review = review.encode('ascii', 'ignore')
        review = review.to_unicode(review)
        review = review.remove_between_square_brackets(review)
        review = review.remove_special_characters(review)
        review = review.lower()
        return review

    def remove_stopwords(self, review, is_lower_case=False):
        # Remove stopwords from the text
        tokens = self.tokenizer.tokenize(review)
        tokens = [token.strip() for token in tokens]
        if is_lower_case:
            filtered_tokens = [token for token in tokens if token not in self.stopwords_list]
        else:
            filtered_tokens = [token for token in tokens if token.lower() not in self.stopwords_list]
        filtered_review = ' '.join(filtered_tokens)
        return filtered_review

    def apply_stemming(self, review):
        # Apply stemming to the text
        return ' '.join([self.stemmer.stem(word) for word in self.tokenizer.tokenize(review)])

    def binarize_sentiment(self, sentiment):
        # Binarize sentiment labels (Positive: 1, Negative: 0)
        return self.lb.fit_transform(sentiment)

    def convert_examples_to_features(self, example_batch):
        # Assuming example_batch is a DataFrame with 'review' and 'sentiment' columns
        df = pd.DataFrame(example_batch)
        df['review'] = df['review'].apply(self.preprocess_text)
        df['review'] = df['review'].apply(self.remove_stopwords)
        df['review'] = df['review'].apply(self.apply_stemming)
        df['sentiment'] = self.binarize_sentiment(df['sentiment'])
        return df
    
    # def convert(self):
    #     data_imdb = load_from_disk(self.config.data_path)
    #     data_imdb_pt = data_imdb.map(self.convert_examples_to_features, batched=True)
    #     data_imdb_pt.save_to_disk(os.path.join(self.config.root_dir,"IMDB_DataSets"))

    def convert(self):
        df = pd.read_csv(self.config.data_path)
        # Preprocess the text and apply stemming and stopword removal (same as before)
        df['review'] = df['review'].apply(self.preprocess_text)
        df['review'] = df['review'].apply(self.remove_stopwords)
        df['review'] = df['review'].apply(self.apply_stemming)
        # Binarize sentiment labels (Positive: 1, Negative: 0)
        df['sentiment'] = self.binarize_sentiment(df['sentiment'])
        
        # Convert the pandas DataFrame to a Hugging Face dataset
        dataset = Dataset.from_pandas(df)
        # Save the processed dataset to disk using Hugging Face's save_to_disk
        dataset.save_to_disk(os.path.join(self.config.root_dir, "IMDB_DataSets"))


In [27]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    
except Exception as e:
    raise e


[2023-07-30 22:40:18,292: INFO: common: YAML file loaded successfully: config\config.yaml]
[2023-07-30 22:40:18,296: INFO: common: YAML file loaded successfully: params.yaml]
[2023-07-30 22:40:18,297: INFO: common: Created directory at: dataStore]
[2023-07-30 22:40:18,297: INFO: common: Created directory at: dataStore/data_transformation]


PermissionError: [Errno 13] Permission denied: 'dataStore/data_ingestion/IMDB_DataSets'

In [17]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import os 

In [18]:
    
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2023-07-30 22:23:54,804: INFO: common: YAML file loaded successfully: config\config.yaml]
[2023-07-30 22:23:54,804: INFO: common: YAML file loaded successfully: params.yaml]
[2023-07-30 22:23:54,804: INFO: common: Created directory at: dataStore]
[2023-07-30 22:23:54,809: INFO: common: Created directory at: dataStore/data_transformation]


FileNotFoundError: [Errno 2] No such file or directory: 'imdb-train.csv'